# NOTEBOOK TO TEST THE BEST MODEL FROM EACH STEP OF THE PROJECT

# Storage

In [15]:
## the path to the `mldl2023` folder in your drive
rootMldl = '<your_drive>/mldl2023'

# Initialization

In [16]:
# packages 1
import shutil
import os
import torch

if not torch.cuda.is_available():
    raise RuntimeError('The model cannot operate without CUDA!')

In [17]:
# mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# packages 2
import random
import string
from typing import Any
from typing import List
import numpy as np
from PIL import Image
from torch import from_numpy
from torchvision.datasets import VisionDataset
import datasets.ss_transforms as tr
from utils.stream_metrics import StreamSegMetrics
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from models import deeplabv3, mobilenetv2
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch, os, copy
import tqdm.notebook as tqdm
import gc
from utils.utils import HardNegativeMining, MeanReduction
import math
import json
import csv
from pprint import pprint
from torch import from_numpy
import datetime
import time

# Memory Management

In [20]:
# garbage collector
def clearCache():
    gc.collect()
    torch.cuda.empty_cache()

# Dataset Class

In [21]:
class IDDADataset(VisionDataset):

    @staticmethod
    def get_mapping():
        classes = [255, 2, 4, 255, 11, 5, 0, 0, 1, 8, 13, 3, 7, 6, 255, 255, 15, 14, 12, 9, 10]
        mapping = 255*np.ones(256, dtype=np.int64)
        mapping[range(len(classes))] = classes
        return lambda x: from_numpy(mapping[x])

    def __init__(self, root, fileNames, transform=None):
        super().__init__(root=root, transform=transform, target_transform=IDDADataset.get_mapping())
        self.fileNames = fileNames

    def __getitem__(self, index):
        image = Image.open(self.root+'/images/'+self.fileNames[index]+'.jpg').convert('RGB')
        label = Image.open(self.root+'/labels/'+self.fileNames[index]+'.png').convert('L')
        if self.transform is not None:
            image, label = self.transform(image, label)
        label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.fileNames)

# Server class

In [22]:
class Server:

    def __init__(self,
                 device, model,
                 datasetTestIdda, datasetTestSame, datasetTestDiff,
                 batchSizeTest,
                 metricClass,
                 rootPretrained):

        self.device = device
        self.model = model

        self.dataLoaderTestIdda = DataLoader(datasetTestIdda, batch_size=batchSizeTest, shuffle=False, drop_last=False)
        self.dataLoaderTestSame = DataLoader(datasetTestSame, batch_size=batchSizeTest, shuffle=False, drop_last=False)
        self.dataLoaderTestDiff = DataLoader(datasetTestDiff, batch_size=batchSizeTest, shuffle=False, drop_last=False)

        self.metricClass = metricClass

        self.rootPretrained = rootPretrained

    def evaluate(self):

        """
        this function evaluates the miou for the best model of each step of the project
        """
        fileNames = []
        for fileName in os.listdir(params.rootPretrained):
            if fileName.endswith('.pth'):
                fileNames.append(fileName)
        fileNames.sort()

        for file_index, fileName in enumerate(fileNames):
            print(f'--------------------- experiment {file_index+1:02d} out of {len(fileNames):02d} ---------------------')

            print(f'-- model: {fileName}')
            clearCache()
            self.model.load_state_dict(torch.load(params.rootPretrained+'/'+fileName))

            start = time.perf_counter()
            miouTestIdda = self.test(self.dataLoaderTestIdda)
            durationTestIdda = int(time.perf_counter() - start)
            print(f'-- miou (test - idda): {100*miouTestIdda:.3f}% -- duration (test - idda): {durationTestIdda}s')

            start = time.perf_counter()
            miouTestSame = self.test(self.dataLoaderTestSame)
            durationTestSame = int(time.perf_counter() - start)
            print(f'-- miou (test - same): {100*miouTestSame:.3f}% -- duration (test - same): {durationTestSame}s')

            start = time.perf_counter()
            miouTestDiff = self.test(self.dataLoaderTestDiff)
            durationTestDiff = int(time.perf_counter() - start)
            print(f'-- miou (test - diff): {100*miouTestDiff:.3f}% -- duration (test - diff): {durationTestDiff}s')

    def test(self, dataLoader):
        clearCache()
        metric = self.metricClass(n_classes=21, name='miou')
        with torch.no_grad():
            self.model.eval()
            pbar = tqdm.tqdm(total=len(dataLoader), desc='testing')
            for batch, (images, labels) in enumerate(dataLoader):
                pbar.set_postfix({
                    'batch': f'{batch+1}/{len(dataLoader)}'
                })
                images = images.to(self.device)
                labels = labels.to(self.device)
                outputs = self.model(images)['out']
                _, prediction = outputs.max(dim=1)
                metric.update(labels.cpu().numpy(), prediction.cpu().numpy())
                pbar.update(1)
        pbar.close()
        miouCum = metric.get_results()['Mean IoU']
        return miouCum

# Different Things

In [23]:
# params class
class Params:
    def __init__(self, **args):
        for key, value in args.items():
            setattr(self, key, value)

In [24]:
# names reader
def getFileNames(root, containerName):
    fileNames = []
    with open(os.path.join(root, containerName), 'r') as file:
        for line in file.read().splitlines():
            fileNames.append(line)
    return fileNames

In [25]:
# main function
def main(params):

    # transformers
    transformsTest = tr.Compose([
        tr.ToTensor(),
        tr.Normalize(tuple(params.transformer_means), tuple(params.transformer_stds))
    ])

    # server
    server = Server(
        device = params.device,
        model = deeplabv3.deeplabv3_mobilenetv2().to(params.device),
        datasetTestIdda = IDDADataset(
            root = params.rootIdda,
            fileNames = getFileNames(params.rootIdda, 'train.txt'),
            transform = transformsTest
        ),
        datasetTestSame = IDDADataset(
            root = params.rootIdda,
            fileNames = getFileNames(params.rootIdda, 'test_same_dom.txt'),
            transform = transformsTest
        ),
        datasetTestDiff = IDDADataset(
            root = params.rootIdda,
            fileNames = getFileNames(params.rootIdda, 'test_diff_dom.txt'),
            transform = transformsTest
        ),
        batchSizeTest = params.batchSizeTest,
        metricClass = StreamSegMetrics,
        rootPretrained = params.rootPretrained
    )

    # evaluate the server
    server.evaluate()

# Server Driver

In [26]:
# params
## the intitial parameters (default config)
## if there's a config value for a parameter, it'll be overwritten
params = Params(
    device = 'cuda:0',
    rootIdda = '/content/data/idda',
    # directory where the best model for each step of the project is stored
    rootPretrained = rootMldl+'/pretrained',
    batchSizeTest = 3,
    transformer_means = [0.320888, 0.292300, 0.288562],
    transformer_stds  = [0.250606, 0.248234, 0.253670],
)

In [27]:
# copies
print('Copying IDDA data ...')
shutil.copytree(rootMldl+'/data/idda', params.rootIdda, dirs_exist_ok=True)

Copying IDDA data ...


'/content/data/idda'

In [28]:
# evaluation
main(params)

--------------------- experiment 01 out of 07 ---------------------
-- model: model_pretrained_step_1.pth


testing:   0%|          | 0/200 [00:00<?, ?it/s]

-- miou (test - idda): 68.143% -- duration (test - idda): 109s


testing:   0%|          | 0/40 [00:00<?, ?it/s]

-- miou (test - same): 62.654% -- duration (test - same): 21s


...

-- miou (test - idda): 67.914% -- duration (test - idda): 108s


testing:   0%|          | 0/40 [00:00<?, ?it/s]

-- miou (test - same): 64.160% -- duration (test - same): 22s


testing:   0%|          | 0/40 [00:00<?, ?it/s]

-- miou (test - diff): 54.551% -- duration (test - diff): 22s
